# Flight Simulator Data Processor

Welcome to the **Flight Simulator Data Processor**! This tool will help you convert your geographical data into a format suitable for use with the **fly-visual** flight simulator.

## Getting Started

1. **Download and run this script**: This script will guide you through processing your data.
2. **Prepare your input file**: Make sure your original file includes the following columns:
   - **Longitude**
   - **Latitude**
   - **Altitude** (in meters)
   - **Timestamp**

## Instructions

- Place your input file in a convenient location and locate the path.
- Run the script to convert your data into the required format.
- Use the output file directly with the flight simulator.

> **Tip:** Double-check that your data is clean and consistent before running the script to avoid errors during processing. In any case, this script will perform some basic checks and cleaning for missing values!

---

In [ ]:
# Run this if you need to install the necessary libraries
!pip install pandas numpy geopy ipywidgets
!pip install meteostat
!pip install requests-cache meteostat
!pip install meteostat geopy tqdm


In [2]:
import math
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from ipywidgets import Button, VBox, Output, FloatText, Text, HTML, Layout, HBox, Label
from IPython.display import display
from meteostat import Stations, Hourly
from datetime import datetime, timedelta
import time
import os
import json
from tqdm import tqdm

In [3]:
# Data processing and cleaning!
# Function to calculate the bearing between two points
def calculate_bearing(lon1, lat1, lon2, lat2):
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    d_lon = lon2 - lon1
    X = math.cos(lat2) * math.sin(d_lon)
    Y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(d_lon)
    
    initial_bearing = math.atan2(X, Y)
    compass_bearing = (math.degrees(initial_bearing) + 360) % 360
    
    return compass_bearing

# Function to add bearing calculations to a dataframe
def add_bearing_to_dataframe(df, lon_col, lat_col):
    bearings = []
    for i in range(len(df) - 1):
        lon1, lat1 = df.iloc[i][lon_col], df.iloc[i][lat_col]
        lon2, lat2 = df.iloc[i + 1][lon_col], df.iloc[i + 1][lat_col]
        bearing = calculate_bearing(lon1, lat1, lon2, lat2)
        bearings.append(bearing)
    bearings.append(None)  # Last point has no next point to calculate bearing
    df['Bearing'] = bearings
    return df

# Function to interpolate latitude, longitude, time, and altitude values based on distance
def interpolate_dataframe(df, distance_interval, lat_col, lon_col, alt_col, time_col):
    Latitudes = df[lat_col].values
    Longitudes = df[lon_col].values
    Altitudes = df[alt_col].values
    Times = pd.to_datetime(df[time_col]).view(np.int64)  # Convert to numeric for interpolation
    
    # Calculate cumulative distance between points
    distances = [0]  # Start with 0 distance for the first point
    for i in range(1, len(Latitudes)):
        point1 = (Latitudes[i-1], Longitudes[i-1])
        point2 = (Latitudes[i], Longitudes[i])
        dist = geodesic(point1, point2).meters
        distances.append(distances[-1] + dist)
    
    # Create new distance points at regular intervals
    total_distance = distances[-1]
    new_distances = np.arange(0, total_distance, distance_interval)
    
    # Interpolate lat/lon/altitude/time based on new distance points
    lat_new = np.interp(new_distances, distances, Latitudes)
    lon_new = np.interp(new_distances, distances, Longitudes)
    alt_new = np.interp(new_distances, distances, Altitudes)
    time_new = np.interp(new_distances, distances, Times)
    
    # Convert interpolated time back to datetime
    time_new = pd.to_datetime(time_new)
    
    # Create new interpolated DataFrame
    interpolated_df = pd.DataFrame({
        'Latitude': lat_new,
        'Longitude': lon_new,
        'Altitude(m)': alt_new,
        'Time': time_new
    })
    
    # Add bearing to the interpolated dataframe
    interpolated_df = add_bearing_to_dataframe(interpolated_df, 'Longitude', 'Latitude')

    return interpolated_df

# Function to save the new dataframe to a CSV file
def save_csv(df, file_path):
    df.to_csv(file_path, index=False)

# Function to handle CSV file loading and processing
def process_csv_file(change):
    # Get the file paths and column names from the input boxes
    input_file_path = file_path_input_text.value
    output_file_path = output_file_path_text.value
    output_file_name = output_file_name_text.value
    
    full_output_path = f"{output_file_path}/{output_file_name}.csv"
    
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(input_file_path)
        output.clear_output()
        
        # Check for missing values and handle them
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            df.fillna(method='ffill', inplace=True)
            with output:
                print(f"Warning: {missing_values} missing values found and filled using the last valid value.")
        
        # Get the column names from the input boxes
        lat_col = lat_col_input_text.value
        lon_col = lon_col_input_text.value
        alt_col = alt_col_input_text.value
        time_col = time_col_input_text.value
        
        # Rename columns to standard names
        df.rename(columns={lat_col: 'Latitude', lon_col: 'Longitude', alt_col: 'Altitude(m)', time_col: 'Time'}, inplace=True)
        
        # Get the distance interval from the input box
        distance_interval = distance_input_text.value
        
        # Interpolate the data and calculate bearings
        interpolated_df = interpolate_dataframe(df, distance_interval=distance_interval, 
                                                lat_col='Latitude', lon_col='Longitude', 
                                                alt_col='Altitude(m)', time_col='Time')
        
        # Save the new data to a CSV file
        save_csv(interpolated_df, full_output_path)
        
        with output:
            print(f"New CSV file with bearing data saved to: {full_output_path}")
    
    except FileNotFoundError:
        output.clear_output()
        with output:
            print("Error: The specified input file was not found. Please check the file path.")
    
    except pd.errors.EmptyDataError:
        output.clear_output()
        with output:
            print("Error: The input CSV file is empty. Please provide a valid file.")
    
    except Exception as e:
        output.clear_output()
        with output:
            print(f"An unexpected error occurred: {e}")

# All the instructions below:
# Layout configuration for better label alignment
label_layout = Layout(width='150px')
text_box_layout = Layout(width='450px')

# Explanatory labels using HTML for formatting
intro_label = HTML("<b>This tool processes a CSV file containing geographical data.</b><br>"
                   "In order to use this tool, you must have longitude, latitude, timestamp, and altitude data.<br>"
                   "You can specify the input and output file paths, column names, and distance interval for interpolation.<br>"
                   "<small><i>Please ensure your altitude values are in meters.</i></small>")

input_label = HTML("<b>Please provide the path to your input CSV file:</b>")
output_label = HTML("<b>Specify where you would like to save the processed output CSV file:</b>")
file_name_label = HTML("<b>Name your output file:</b>")

column_label = HTML("<b>Provide the current column names in your CSV file for the following fields (case sensitive):</b>")
distance_label = HTML("<b>Set the distance interval (in meters) for data interpolation:</b><br>"
                      "<small>A lower value produces a smoother flythrough, but may increase file size that the simulator is unable to process.</small>")

# Create input boxes for file paths and output file name
file_path_input_text = Text(layout=text_box_layout)
file_path_input = HBox([Label("Input Directory:", layout=label_layout), file_path_input_text])

output_file_path_text = Text(layout=text_box_layout)
output_file_path_input = HBox([Label("Output Directory:", layout=label_layout), output_file_path_text])

output_file_name_text = Text(layout=text_box_layout)
output_file_name_input = HBox([Label("Output File Name:", layout=label_layout), output_file_name_text])

# Create input boxes for custom column names
lat_col_input_text = Text(value='Latitude', layout=text_box_layout)
lat_col_input = HBox([Label("Latitude Column:", layout=label_layout), lat_col_input_text])

lon_col_input_text = Text(value='Longitude', layout=text_box_layout)
lon_col_input = HBox([Label("Longitude Column:", layout=label_layout), lon_col_input_text])

alt_col_input_text = Text(value='Altitude(m)', layout=text_box_layout)
alt_col_input = HBox([Label("Altitude Column:", layout=label_layout), alt_col_input_text])

time_col_input_text = Text(value='Time', layout=text_box_layout)
time_col_input = HBox([Label("Time Column:", layout=label_layout), time_col_input_text])

# Create a distance input box
distance_input_text = FloatText(value=50.0, step=0.1, layout=text_box_layout)
distance_input = HBox([Label("Distance Interval (m):", layout=label_layout), distance_input_text])

# Create a button to process the file
process_button = Button(description="Process File")
process_button.on_click(process_csv_file)

# Output widget to display messages
output = Output()

# Display the widgets with explanatory labels and better formatting
display(VBox([intro_label, 
              input_label, file_path_input, 
              output_label, output_file_path_input, 
              file_name_label, output_file_name_input, 
              column_label, lat_col_input, lon_col_input, alt_col_input, time_col_input, 
              distance_label, distance_input, 
              process_button, output]))


In [3]:

# Cache directory
CACHE_DIR = 'meteostat_cache'
if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

# Weather code to condition mapping
WEATHER_CODE_MAPPING = {
    1: 'Clear',
    2: 'Fair',
    3: 'Cloudy',
    4: 'Overcast',
    5: 'Fog',
    6: 'Freezing Fog',
    7: 'Light Rain',
    8: 'Rain',
    9: 'Heavy Rain',
    10: 'Freezing Rain',
    11: 'Heavy Freezing Rain',
    12: 'Sleet',
    13: 'Heavy Sleet',
    14: 'Light Snowfall',
    15: 'Snowfall',
    16: 'Heavy Snowfall',
    17: 'Rain Shower',
    18: 'Heavy Rain Shower',
    19: 'Sleet Shower',
    20: 'Heavy Sleet Shower',
    21: 'Snow Shower',
    22: 'Heavy Snow Shower',
    23: 'Lightning',
    24: 'Hail',
    25: 'Thunderstorm',
    26: 'Heavy Thunderstorm',
    27: 'Storm'
}

# Function to find the nearest weather station
def get_nearest_station(lat, lon):
    stations = Stations().nearby(lat, lon).fetch(1)
    if not stations.empty:
        return stations.index[0]
    return None

# Function to calculate the bearing between two points
def calculate_bearing(lon1, lat1, lon2, lat2):
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    d_lon = lon2 - lon1
    X = math.cos(lat2) * math.sin(d_lon)
    Y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(d_lon)
    
    initial_bearing = math.atan2(X, Y)
    compass_bearing = (math.degrees(initial_bearing) + 360) % 360
    
    return compass_bearing

# Function to calculate velocity between two points
def calculate_velocity(dist1, time1, dist2, time2):
    time_diff = (time2 - time1).total_seconds()
    if time_diff == 0:
        return 0
    distance_diff = dist2 - dist1
    return distance_diff / time_diff

# Function to calculate acceleration between two points
def calculate_acceleration(vel1, time1, vel2, time2):
    time_diff = (time2 - time1).total_seconds()
    if time_diff == 0:
        return 0
    velocity_diff = vel2 - vel1
    return velocity_diff / time_diff

# Function to add bearing, velocity, and acceleration calculations to a dataframe
def add_motion_metrics_to_dataframe(df, lon_col, lat_col, time_col):
    bearings = []
    velocities = []
    accelerations = []

    for i in range(len(df) - 1):
        lon1, lat1, time1 = df.iloc[i][lon_col], df.iloc[i][lat_col], pd.to_datetime(df.iloc[i][time_col])
        lon2, lat2, time2 = df.iloc[i + 1][lon_col], df.iloc[i + 1][lat_col], pd.to_datetime(df.iloc[i + 1][time_col])
        
        # Calculate bearing
        bearing = calculate_bearing(lon1, lat1, lon2, lat2)
        bearings.append(bearing)
        
        # Calculate velocity
        dist1 = geodesic((lat1, lon1), (lat2, lon2)).meters
        velocity = calculate_velocity(0, time1, dist1, time2)
        velocities.append(velocity)
        
        # Calculate acceleration
        if i > 0:
            prev_velocity = velocities[i - 1]
            acceleration = calculate_acceleration(prev_velocity, time1, velocity, time2)
        else:
            acceleration = 0  # First data point has no previous acceleration
        
        accelerations.append(acceleration)
    
    # Add last None for bearing, velocity, and acceleration as they can't be calculated for the last point
    bearings.append(None)
    velocities.append(None)
    accelerations.append(None)
    
    df['Bearing'] = bearings
    df['Velocity(m/s)'] = velocities
    df['Acceleration(m/s²)'] = accelerations
    return df

def fetch_weather_data(lat, lon, timestamp):
    # Get the nearest weather station ID
    station_id = get_nearest_station(lat, lon)
    if not station_id:
        print(f"No weather station found near lat: {lat}, lon: {lon}")
        return {
            'wind_speed': None,
            'temperature': None,
            'precipitation': None,
            'wind_direction': None,
            'wind_gust': None,
            'humidity': None,
            'weather_code': None
        }

    cache_key = f"{station_id}_{timestamp.strftime('%Y-%m-%dT%H:%M')}.json"
    cache_path = os.path.join(CACHE_DIR, cache_key)
    
    # Check if data is in cache
    if os.path.exists(cache_path):
        with open(cache_path, 'r') as f:
            data = json.load(f)
        return data
    
    # Fetch data from Meteostat for the nearest station
    data = Hourly(station_id, start=timestamp, end=timestamp + timedelta(hours=1)).fetch()

    if data.empty:
        print(f"No data returned from API for station {station_id} at {timestamp}")
        return {
            'wind_speed': None,
            'temperature': None,
            'precipitation': None,
            'wind_direction': None,
            'wind_gust': None,
            'humidity': None,
            'weather_code': None
        }
    
    # Debugging: Print the available keys in the fetched data
    print(f"Data retrieved for station {station_id} at {timestamp}:")
    print(data.head())  # Print first row of data to inspect what was fetched
    
    # Extract weather data
    weather_data = {
        'wind_speed': data.iloc[0]['wspd'] if 'wspd' in data.columns else None,
        'temperature': data.iloc[0]['temp'] if 'temp' in data.columns else None,
        'precipitation': data.iloc[0]['prcp'] if 'prcp' in data.columns else None,
        'wind_direction': data.iloc[0]['wdir'] if 'wdir' in data.columns else None,
        'wind_gust': data.iloc[0]['wpgt'] if 'wpgt' in data.columns else None,
        'humidity': data.iloc[0]['rhum'] if 'rhum' in data.columns else None,
        'weather_code': data.iloc[0]['coco'] if 'coco' in data.columns else None
    }
    
    # Log missing fields for debugging
    for key, value in weather_data.items():
        if value is None:
            print(f"Warning: {key} is missing for station {station_id} at {timestamp}")
    
    # Save to cache
    with open(cache_path, 'w') as f:
        json.dump(weather_data, f)
    
    # Pause to avoid hitting API rate limits
    time.sleep(0.005)
    
    return weather_data

# Function to add weather data to the dataframe
def add_weather_data_to_dataframe(df, lat_col, lon_col, time_col):
    weather_data_list = {
        'Wind Speed (m/s)': [],
        'Temperature (°C)': [],
        'Precipitation (mm)': [],
        'Wind Direction (°)': [],
        'Wind Gust (m/s)': [],
        'Humidity (%)': [],
        'Weather Code': []
    }
    
    for i in tqdm(range(0, len(df), 10)):  # Only process every 10th row
        lat, lon, timestamp = df.iloc[i][lat_col], df.iloc[i][lon_col], pd.to_datetime(df.iloc[i][time_col])
        weather_data = fetch_weather_data(lat, lon, timestamp)
        
        # Append the data to the lists, ensuring all keys are present
        weather_data_list['Wind Speed (m/s)'].append(weather_data.get('wind_speed'))
        weather_data_list['Temperature (°C)'].append(weather_data.get('temperature'))
        weather_data_list['Precipitation (mm)'].append(weather_data.get('precipitation'))
        weather_data_list['Wind Direction (°)'].append(weather_data.get('wind_direction'))
        weather_data_list['Wind Gust (m/s)'].append(weather_data.get('wind_gust'))
        weather_data_list['Humidity (%)'].append(weather_data.get('humidity'))
        weather_data_list['Weather Code'].append(weather_data.get('weather_code'))
    
    # Interpolate weather data for all rows
    for key, values in weather_data_list.items():
        # Convert list to pandas Series, ensuring correct length by reindexing
        series = pd.Series(values, index=df.index[::10]).interpolate(method='linear', limit_direction='forward').reindex(df.index, method='nearest').fillna(method='ffill')
        df[key] = series
    
    # Map weather codes to weather conditions
    df['Weather Condition'] = df['Weather Code'].map(WEATHER_CODE_MAPPING)
    
    return df

# Function to interpolate latitude, longitude, time, and altitude values based on distance
def interpolate_dataframe(df, distance_interval, lat_col, lon_col, alt_col, time_col):
    Latitudes = df[lat_col].values
    Longitudes = df[lon_col].values
    Altitudes = df[alt_col].values
    Times = pd.to_datetime(df[time_col]).view(np.int64)  # Convert to numeric for interpolation
    
    # Calculate cumulative distance between points
    distances = [0]  # Start with 0 distance for the first point
    for i in range(1, len(Latitudes)):
        point1 = (Latitudes[i-1], Longitudes[i-1])
        point2 = (Latitudes[i], Longitudes[i])
        dist = geodesic(point1, point2).meters
        distances.append(distances[-1] + dist)
    
    # Create new distance points at regular intervals
    total_distance = distances[-1]
    new_distances = np.arange(0, total_distance, distance_interval)
    
    # Interpolate lat/lon/altitude/time based on new distance points
    lat_new = np.interp(new_distances, distances, Latitudes)
    lon_new = np.interp(new_distances, distances, Longitudes)
    alt_new = np.interp(new_distances, distances, Altitudes)
    time_new = np.interp(new_distances, distances, Times)
    
    # Convert interpolated time back to datetime
    time_new = pd.to_datetime(time_new)
    
    # Create new interpolated DataFrame
    interpolated_df = pd.DataFrame({
        'Latitude': lat_new,
        'Longitude': lon_new,
        'Altitude(m)': alt_new,
        'Time': time_new
    })
    
    # Add motion metrics to the interpolated dataframe
    interpolated_df = add_motion_metrics_to_dataframe(interpolated_df, 'Longitude', 'Latitude', 'Time')
    
    # Add weather data to the interpolated dataframe
    interpolated_df = add_weather_data_to_dataframe(interpolated_df, 'Latitude', 'Longitude', 'Time')

    return interpolated_df

# Function to save the new dataframe to a CSV file
def save_csv(df, file_path):
    df.to_csv(file_path, index=False)

# Function to handle CSV file loading and processing
def process_csv_file(change):
    # Get the file paths and column names from the input boxes
    input_file_path = file_path_input_text.value
    output_file_path = output_file_path_text.value
    output_file_name = output_file_name_text.value
    
    full_output_path = f"{output_file_path}/{output_file_name}.csv"
    
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(input_file_path)
        output.clear_output()
        
        # Check for missing values and handle them
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            df.fillna(method='ffill', inplace=True)
            with output:
                print(f"Warning: {missing_values} missing values found and filled using the last valid value.")
        
        # Get the column names from the input boxes
        lat_col = lat_col_input_text.value
        lon_col = lon_col_input_text.value
        alt_col = alt_col_input_text.value
        time_col = time_col_input_text.value
        
        # Rename columns to standard names
        df.rename(columns={lat_col: 'Latitude', lon_col: 'Longitude', alt_col: 'Altitude(m)', time_col: 'Time'}, inplace=True)
        
        # Get the distance interval from the input box
        distance_interval = distance_input_text.value
        
        # Interpolate the data and calculate bearings, velocity, and acceleration
        interpolated_df = interpolate_dataframe(df, distance_interval=distance_interval, 
                                                lat_col='Latitude', lon_col='Longitude', 
                                                alt_col='Altitude(m)', time_col='Time')
        
        # Save the new data to a CSV file
        save_csv(interpolated_df, full_output_path)
        
        with output:
            print(f"New CSV file with bearing, velocity, acceleration, and weather data saved to: {full_output_path}")
    
    except FileNotFoundError:
        output.clear_output()
        with output:
            print("Error: The specified input file was not found. Please check the file path.")
    
    except pd.errors.EmptyDataError:
        output.clear_output()
        with output:
            print("Error: The input CSV file is empty. Please provide a valid file.")
    
    except Exception as e:
        output.clear_output()
        with output:
            print(f"An unexpected error occurred: {e}")

# Layout configuration for better label alignment
label_layout = Layout(width='150px')
text_box_layout = Layout(width='450px')

# Explanatory labels using HTML for formatting
intro_label = HTML("<b>This tool processes a CSV file containing geographical data.</b><br>"
                   "In order to use this tool, you must have longitude, latitude, timestamp, and altitude data.<br>"
                   "You can specify the input and output file paths, column names, and distance interval for interpolation.<br>"
                   "<small><i>Please ensure your altitude values are in meters.</i></small>")

input_label = HTML("<b>Please provide the path to your input CSV file:</b>")
output_label = HTML("<b>Specify where you would like to save the processed output CSV file:</b>")
file_name_label = HTML("<b>Name your output file:</b>")

column_label = HTML("<b>Provide the current column names in your CSV file for the following fields (case sensitive):</b>")
distance_label = HTML("<b>Set the distance interval (in meters) for data interpolation:</b><br>"
                      "<small>A lower value produces a smoother flythrough, but may increase file size that the simulator is unable to process.</small>")

# Create input boxes for file paths and output file name
file_path_input_text = Text(layout=text_box_layout)
file_path_input = HBox([Label("Input Directory:", layout=label_layout), file_path_input_text])

output_file_path_text = Text(layout=text_box_layout)
output_file_path_input = HBox([Label("Output Directory:", layout=label_layout), output_file_path_text])

output_file_name_text = Text(layout=text_box_layout)
output_file_name_input = HBox([Label("Output File Name:", layout=label_layout), output_file_name_text])

# Create input boxes for custom column names
lat_col_input_text = Text(value='Latitude', layout=text_box_layout)
lat_col_input = HBox([Label("Latitude Column:", layout=label_layout), lat_col_input_text])

lon_col_input_text = Text(value='Longitude', layout=text_box_layout)
lon_col_input = HBox([Label("Longitude Column:", layout=label_layout), lon_col_input_text])

alt_col_input_text = Text(value='Altitude(m)', layout=text_box_layout)
alt_col_input = HBox([Label("Altitude Column:", layout=label_layout), alt_col_input_text])

time_col_input_text = Text(value='Time', layout=text_box_layout)
time_col_input = HBox([Label("Time Column:", layout=label_layout), time_col_input_text])

# Create a distance input box
distance_input_text = FloatText(value=50.0, step=0.1, layout=text_box_layout)
distance_input = HBox([Label("Distance Interval (m):", layout=label_layout), distance_input_text])

# Create a button to process the file
process_button = Button(description="Process File")
process_button.on_click(process_csv_file)

# Output widget to display messages
output = Output()

# Display the widgets with explanatory labels and better formatting
display(VBox([intro_label, 
              input_label, file_path_input, 
              output_label, output_file_path_input, 
              file_name_label, output_file_name_input, 
              column_label, lat_col_input, lon_col_input, alt_col_input, time_col_input, 
              distance_label, distance_input, 
              process_button, output]))
